In [3]:
def translator_dictionary():

    query3 = "SELECT movieId, imdbId FROM links"
    df_translator = pd.read_sql(query3, db)
    movie_IDs = list(df_translator['movieId'])
    IMDB_IDs = list(df_translator['imdbId'])


    ML_2_IMDB = dict(zip(movie_IDs, IMDB_IDs))
    IMDB_2_ML = dict(zip(IMDB_IDs, movie_IDs))
    
    return IMDB_2_ML

def convert_django(dataframe, django_data):
    
    new_user_row = pd.DataFrame(np.zeros(shape=(1,len(dataframe.columns))),
                              columns=dataframe.columns)

    dfs_to_concat = [dataframe, new_user_row]
    combined = pd.concat(dfs_to_concat)
    
    translator = translator_dictionary()
    
    #Assuming that data coming from Django interface is a list of tuples
    for pair in django_data:
        movieId = translator[pair[0]]
        combined.loc[0,('rating', movieId)] = pair[1] 
        #where 0 represents userId = 0, aka the new user
        
    return list(combined.loc[0])

In [ ]:
query2 = "SELECT * FROM ratings"
df_sandbox = pd.read_sql(query2, db)
df_sandbox = df_sandbox.drop("timestamp", axis = 1)
df_sandbox = df_sandbox.set_index(["userId", "movieId"])
users_vs_movies_matrix = df_sandbox.unstack()
users_vs_movies_matrix = users_vs_movies_matrix.fillna(0)

In [2]:
django_test = [(114709, 4.0), (114885, 5.0), (114319, 1.0), (112302, 3.0), (3837248, 3.0)]

In [ ]:

new_user_row = pd.DataFrame(np.zeros(shape=(1,len(users_vs_movies_matrix.columns))),
                              columns=users_vs_movies_matrix.columns)

dfs_to_concat = [users_vs_movies_matrix, new_user_row]
combined = pd.concat(dfs_to_concat)

combined.tail()